In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd 
DATASET = "amazon_reviews_us_Grocery_v1_00.tsv"
import logging
import warnings
import sys
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS
import gensim

warnings.simplefilter('ignore')
logging.disable(sys.maxsize)



In [3]:
import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

C:\Users\geoio\Anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [4]:
dataset = pd.read_table(DATASET, error_bad_lines=False, header=0, warn_bad_lines=False) 
# there are some malformed entries the dataset... let's ignore those for now


In [5]:
# Lots of spammy duplicate reviews...might be worth checking if the same person is duplicating the reviews across multiple products

# Additions, because maybe same review_body is not duplicate i.e. "Good" as review.
x = dataset[dataset['review_body'].duplicated() == True]
x = x[dataset['customer_id'].duplicated() == True] 
x = x[dataset['product_id'].duplicated() == True] 
#print(len(x))
#x.head()

In [6]:
#remove NaNs from the dataset  for now...
dataset = dataset.dropna()

In [7]:
dataset.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [8]:
reviews_body = dataset["review_body"]
reviews_head = dataset["review_headline"]
#print(reviews.head())
#print(sorted_negative_rows["review_headline"])

In [9]:
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if (token not in gensim.parsing.preprocessing.STOPWORDS): 
            if (WordNetLemmatizer().lemmatize(token, 'v') != "star"): # Remove star to remove reviews such as: "Five stars"
                result.append(WordNetLemmatizer().lemmatize(token, 'v')) # for now, lemmatizing only verbs
    return result

In [10]:
for review in reviews_head[8:10]:
    print("Removed stopwords and lemmatized words")
    print(review + " becomes -->", preprocess(review))

Removed stopwords and lemmatized words
Great tasting! becomes --> ['great', 'taste']
Removed stopwords and lemmatized words
Disgusting now and difficult on digestion becomes --> ['disgust', 'difficult', 'digestion']


In [11]:
def perform_topic_modelling(reviews):
    processed = []
    for review in reviews:
        processed.append(preprocess(review))
    dictionary = gensim.corpora.Dictionary(processed) # construct word<->id mappings - it does it in alphabetical order
    bow_corpus = [dictionary.doc2bow(review) for review in processed]
    lda_model =  gensim.models.LdaModel(bow_corpus, 
                                   num_topics=20, 
                                   id2word=dictionary,                                    
                                   passes=10,
                                   random_state = 900,
                                   ) 
    return lda_model, bow_corpus, dictionary

In [12]:
less_reviews_body = reviews_body[:1000]
lda_model_body, bow_corpus, dictionary = perform_topic_modelling(less_reviews_body)

pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_body, bow_corpus, dictionary)

LDAvis_prepared


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.037104 -0.062690       1        1  16.555248
5      0.127963 -0.061334       2        1   8.492292
17     0.098540 -0.058237       3        1   6.676447
4      0.021582 -0.078923       4        1   6.194984
11     0.011124  0.018771       5        1   6.046015
3      0.029903 -0.026897       6        1   5.714346
18     0.073663  0.063534       7        1   5.365588
12    -0.139454 -0.115897       8        1   5.293592
0      0.026187 -0.046075       9        1   5.036093
8     -0.013539  0.004924      10        1   4.531425
1      0.059884  0.012994      11        1   4.093443
7     -0.047019 -0.024770      12        1   3.730499
2      0.002394  0.028147      13        1   3.438899
16    -0.122046 -0.081442      14        1   3.337333
6      0.027970  0.082996      15        1   3.306622
19     0.003678  0.007611      16        1   3.032060
9     -0.025098  0.121076      17        1   2.940978
14    -0.041414  0.044745      18        1   2.915452
10    -0.035890  0.094166      19        1   1.983991
13    -0.095532  0.077302      20        1   1.314693, topic_info=      Term        Freq       Total Category  logprob  loglift
152     br  198.000000  198.000000  Default  30.0000  30.0000
18   great  214.000000  214.000000  Default  29.0000  29.0000
21    love  183.000000  183.000000  Default  28.0000  28.0000
44    good  176.000000  176.000000  Default  27.0000  27.0000
47     tea   78.000000   78.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
425   soup    0.643883   11.977460  Topic20  -5.6021   1.4083
185  think    0.643182   31.565627  Topic20  -5.6032   0.4382
257   read    0.643129   11.193134  Topic20  -5.6033   1.4749
75     buy    0.642770  107.035799  Topic20  -5.6038  -0.7836
166   kind    0.642475   15.208309  Topic20  -5.6043   1.1673

[1406 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2319     10  0.621125        abba
1725     17  0.858813     ability
511       8  0.402544    absolute
511      11  0.402544    absolute
249       1  0.241281  absolutely
...     ...       ...         ...
389      12  0.493348       yummy
389      13  0.044850       yummy
389      14  0.089700       yummy
2322     10  0.621124      zabbas
1724      1  0.892120    zucchini

[3162 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 6, 18, 5, 12, 4, 19, 13, 1, 9, 2, 8, 3, 17, 7, 20, 10, 15, 11, 14])

In [13]:
### I will run this on the first 1000 reviews because for now, because it will take long for more
less_reviews_head = reviews_head[:1000]
lda_model_head, bow_corpus, dictionary = perform_topic_modelling(less_reviews_head)
#print(processed) # All the words after applying lemmatization on the first 1000 reviews

In [14]:
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_head, bow_corpus, dictionary)

LDAvis_prepared


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15    -0.221547 -0.042048       1        1  7.862667
16    -0.045624 -0.009435       2        1  6.816124
2      0.021662  0.261715       3        1  6.428772
12    -0.202159 -0.086655       4        1  5.834338
17     0.088291  0.014043       5        1  5.677151
4      0.126956 -0.120415       6        1  5.639374
3     -0.057321  0.044819       7        1  5.635285
13    -0.083197 -0.037358       8        1  5.458608
1      0.018038  0.093498       9        1  5.250571
5      0.028911 -0.046703      10        1  4.642924
6     -0.036906  0.114763      11        1  4.634566
10     0.063134 -0.011548      12        1  4.607526
11     0.032405 -0.001156      13        1  4.244294
19    -0.074813  0.015591      14        1  4.150783
9      0.007241  0.002621      15        1  4.117894
18     0.038821 -0.029125      16        1  4.017451
8      0.057619 -0.056026      17        1  3.986497
7      0.049736 -0.026546      18        1  3.867670
14     0.103941 -0.059841      19        1  3.771419
0      0.084814 -0.020195      20        1  3.356087, topic_info=          Term       Freq      Total Category  logprob  loglift
0         love  38.000000  38.000000  Default  30.0000  30.0000
24        good  52.000000  52.000000  Default  29.0000  29.0000
4        great  84.000000  84.000000  Default  28.0000  28.0000
47     product  24.000000  24.000000  Default  27.0000  27.0000
21        best  25.000000  25.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
476      hubby   0.705106   2.123642  Topic20  -4.5403   2.2919
77         bar   0.705105   5.723474  Topic20  -4.5403   1.3004
18   delicious   0.705105  12.502636  Topic20  -4.5403   0.5190
345      blend   0.705104   2.143494  Topic20  -4.5403   2.2825
268     season   0.705104   2.757686  Topic20  -4.5403   2.0306

[952 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
451      13  0.698572  absolutely
628       2  0.688654        ache
93       14  0.714061    actually
290      17  0.481098         add
290      18  0.481098         add
...     ...       ...         ...
14       12  0.112361       yummy
14       18  0.786527       yummy
322      11  0.469324       zevia
322      16  0.469324       zevia
678       5  0.718191       zingy

[1159 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 17, 3, 13, 18, 5, 4, 14, 2, 6, 7, 11, 12, 20, 10, 19, 9, 8, 15, 1])

In [15]:
def unseen_document(review, lda_model):  
    print(review)
    preprocess(review)
    bow_vector = dictionary.doc2bow(preprocess(review))
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        print("Index: {}\nScore: {}\t Topic: {}".format(index, score, lda_model.print_topic(index, 5)))
    #print(lda_model[bow_vector]) # How likely each class is. Because the review is small, some classes have very low probabilities.

In [16]:
unseen_document(reviews_head[1005], lda_model_head)

Great cream!
Index: 15
Score: 0.3500105142593384	 Topic: 0.249*"great" + 0.093*"taste" + 0.042*"coffee" + 0.028*"thank" + 0.026*"fresh"
Index: 11
Score: 0.3499799072742462	 Topic: 0.061*"like" + 0.035*"ok" + 0.026*"stale" + 0.026*"amaze" + 0.018*"wood"
Index: 0
Score: 0.0166671983897686	 Topic: 0.082*"disappoint" + 0.031*"order" + 0.031*"perfect" + 0.021*"make" + 0.021*"work"
Index: 1
Score: 0.0166671983897686	 Topic: 0.056*"good" + 0.040*"pretty" + 0.031*"flavor" + 0.020*"pack" + 0.016*"product"
Index: 2
Score: 0.0166671983897686	 Topic: 0.281*"good" + 0.074*"tea" + 0.046*"taste" + 0.028*"stuff" + 0.024*"enjoy"
Index: 3
Score: 0.0166671983897686	 Topic: 0.054*"great" + 0.033*"price" + 0.031*"quality" + 0.027*"flavor" + 0.026*"better"
Index: 4
Score: 0.0166671983897686	 Topic: 0.175*"best" + 0.028*"flavor" + 0.024*"love" + 0.020*"melt" + 0.020*"mix"
Index: 5
Score: 0.0166671983897686	 Topic: 0.081*"flavor" + 0.025*"day" + 0.021*"make" + 0.021*"perfect" + 0.017*"tasteless"
Index: 6
Scor

In [17]:
example_topic = less_reviews_body[322]
print(example_topic)
bow_vector = dictionary.doc2bow(preprocess(example_topic))
for index, score in sorted(lda_model_body[bow_vector], key=lambda tup: -1*tup[1]):
    print("Index: {}\nScore: {}\t Topic: {}".format(index, score, lda_model_body.print_topic(index, 10)))

The marshmallow bags were difficult to open.  The marshmallows were melted and stuck not only to themselves but the bags.  These were meant to be added to a snack mix for a child's birthday party.  I was unable to use them because they were one/a few big globs of marshmallow and because of that they did not resemble ice cream cones nor did they work for the snack mix.  Very disappointed.
Index: 17
Score: 0.2799563705921173	 Topic: 0.032*"taste" + 0.026*"good" + 0.024*"like" + 0.018*"buy" + 0.017*"tea" + 0.016*"great" + 0.015*"love" + 0.012*"regular" + 0.011*"little" + 0.011*"product"
Index: 8
Score: 0.24314114451408386	 Topic: 0.022*"perfect" + 0.020*"taste" + 0.017*"order" + 0.016*"good" + 0.014*"time" + 0.013*"snack" + 0.013*"need" + 0.012*"great" + 0.011*"like" + 0.011*"eat"
Index: 11
Score: 0.1815846562385559	 Topic: 0.027*"good" + 0.024*"flavor" + 0.015*"protein" + 0.012*"buy" + 0.011*"sauce" + 0.010*"br" + 0.010*"like" + 0.010*"soy" + 0.010*"taste" + 0.009*"better"
Index: 14
Scor

In [18]:
example_topic = less_reviews_head[322]
print(example_topic)
bow_vector = dictionary.doc2bow(preprocess(example_topic))
for index, score in sorted(lda_model_head[bow_vector], key=lambda tup: -1*tup[1]):
    print("Index: {}\nScore: {}\t Topic: {}".format(index, score, lda_model_head.print_topic(index, 5)))



Disappointed
Index: 0
Score: 0.5249990224838257	 Topic: 0.082*"disappoint" + 0.031*"order" + 0.031*"perfect" + 0.021*"make" + 0.021*"work"
Index: 1
Score: 0.02500004880130291	 Topic: 0.056*"good" + 0.040*"pretty" + 0.031*"flavor" + 0.020*"pack" + 0.016*"product"
Index: 2
Score: 0.02500004880130291	 Topic: 0.281*"good" + 0.074*"tea" + 0.046*"taste" + 0.028*"stuff" + 0.024*"enjoy"
Index: 3
Score: 0.02500004880130291	 Topic: 0.054*"great" + 0.033*"price" + 0.031*"quality" + 0.027*"flavor" + 0.026*"better"
Index: 4
Score: 0.02500004880130291	 Topic: 0.175*"best" + 0.028*"flavor" + 0.024*"love" + 0.020*"melt" + 0.020*"mix"
Index: 5
Score: 0.02500004880130291	 Topic: 0.081*"flavor" + 0.025*"day" + 0.021*"make" + 0.021*"perfect" + 0.017*"tasteless"
Index: 6
Score: 0.02500004880130291	 Topic: 0.068*"good" + 0.044*"great" + 0.029*"try" + 0.024*"flavor" + 0.024*"value"
Index: 7
Score: 0.02500004880130291	 Topic: 0.092*"yummy" + 0.053*"excellent" + 0.027*"satisfy" + 0.018*"chocolate" + 0.018*"whi

Example that Header of review is better than body: <br>
<em>Review head</em>: Dissapoint <br>
<em>Review body</em>: The marshmallow bags were difficult to open.  The marshmallows were melted and stuck not only to themselves but the bags.  These were meant to be added to a snack mix for a child's birthday party.  I was unable to use them because they were one/a few big globs of marshmallow and because of that they did not resemble ice cream cones nor did they work for the snack mix.  Very disappointed. <br>
<em>Topic by running model on head</em>: 0.082*"disappoint" + 0.031*"order" + 0.031*"perfect" + 0.021*"make" + 0.021*"work" <br>
<em>Topic by running model on body</em>: Topic: 0.019*"love" + 0.010*"good" + 0.010*"add" + 0.010*"nice" + 0.010*"pasta" + 0.010*"tea" + 0.009*"best" + 0.008*"snack" + 0.007*"health" + 0.007*"healthy" (MAYBE because of add) <br>
By running on body, no bad Topic categories were found!!
